<a href="https://colab.research.google.com/github/henricobela/cyber_cp1/blob/main/Imigration_Verifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sejam bem-vindos a minha API para INTERPOL!
####Meu Nome é Henrico, sou aluno da FIAP (RM95985)
--------------------------------------------------------------------------------
#Vamos começar?

####Esta aplicação trata-se de uma pagina na web criada com streamlit, utilizando a linguagem python, fuzzywuzzy, ngrok e regex. Ela faz uma verificação na base de dados da Interpol (fictícia), e uma verificação na base de dados de passaportes proibidos da França (fictícia). Com essa verificação é definida a entrada ou não de uma certa pessoa á França!
--------------------------------------------------------------------------------

#####Para começar

#####Primeiro vamos instalar todos os recursos necessários para a realização da API, instalando FuzzyWuzzy, Ngrok, Streamlit/ace.

--------------------------------------------------------------------------------


#1º Instalando recursos

In [37]:
!pip install -q streamlit

In [38]:
!pip install -q streamlit-ace

In [39]:
!pip install -q fuzzywuzzy[speedup]

In [40]:
!pip install -q pyngrok

##2º - Criando aplicativo do streamlit para gerar a pagina web


In [41]:
!streamlit run app.py &>/dev/null&

##3º - Importando a biblioteca NGROK

In [42]:
from pyngrok import ngrok

##4º Aplicação


In [51]:
#criando o arquivo de aplicação .py
%%writefile app.py  
import streamlit as st                                                                              #importando streamlit e setando como st
import re                                                                                           #importando o regex
from fuzzywuzzy import process                                                                      #importando process do fuzzy
import json                                                                                         #importando json
import requests as r                                                                                #importando requests


def main():                                                                                         #define a função que cria webpage
  nameData = r.get("https://cspinheiro.github.io/interpol.json")                                    #consome json dos nomes da interpol e guarda na variavel nameDataJson
  passportData = r.get("https://henricobela.github.io/fra.json")                                    #consome json da França com numeros de passaportes e salva na variavel fraDataJson



  def verifyNamesInterpol(names):                                                                   #define a função de verificação dos nomes no endpoint da interpol
    invalidNames = nameData.json()["interpol"]                                                      #salva na variavel invalidNames tudo que tiver interpol no arquivo json
    invalid = []                                                                                    #lista invalid vazia

    extraction = process.extract(names, invalidNames)                                               #salva na variavel extraction, utilizando o fuzzy para extrair os nomes do endpoint

    for x in extraction:                                                                            #laço for para cada elemento "x" na variavel extraction - faça:
      if x[1] >= 76:                                                                                #condição if para threshold acima ou igual da porcentagem de 80
        invalid.append(x)                                                                           #acrescenta na lista vazia invalid o elemento "x"
      if len(invalid) == 0:                                                                         #condição if para o comprimento da lista invalid comparando se for 0
        return "Allow"                                                                              #resultado da comparação se for verdadeira retorna internamente "Allow"
      return "Deny"                                                                                 #se o resultado da condição for falso, retorna internamente "Deny"



  def verifyPassportNmbr(passportNmbr):                                                             #define a função que verifica os numeros de passaporte    
    country = re.search(r"([\D]{3})", passportNmbr.lower())                                         #guarda na variavel country a busca pelos elementos dentro das letras digitadas pelo usuario
    passport = re.search("(?i)(\d+)", passportNmbr.lower())                                         #guarda na variavel passport a busca pelos elementos dentro do numero de passaporte digitado pelo usuario
    country = str(country[0])                                                                       #guarda na variavel c, a primeira posição, no caso as letras do país
                                                                                    
    if country and passport:                                                                        #condição se encontrar country e passport - faça:
      endpoint = passportData.json()[country]                                                       #salva na variavel endpoint tudo que tiver country letras no arquivo json
      for item in endpoint:                                                                         #laço for para cada item dentro da variavel dict_api - faça:
        for value in item.values():                                                                 #laço for para cada valor dentro dos valores do arquivo json - faça:
          if str(passportNmbr) == str(country) + str(value):                                        #condição que compara se o numero de passaporte digitado é igual as letras + os valores que existem no arquivo json
            return "Deny"                                                                           #resultado da comparação se for falsa retorna internamente "Deny"
      return "Allow"                                                                                #resultado da condição for verdadeira, retorna internamente "Allow"



  def verifyPerson(name, passportNmbr):                                                             #define a função que verifica o nome e o passaporte digitado
    person = verifyNamesInterpol(name)                                                              #guarda na variavel person o nome digitado
    passport = verifyPassportNmbr(passportNmbr)                                                     #guarda na variavel passport o passaporte digitado

    if passport == "Deny" or person == "Deny":                                                      #condição que diz se o passaporte for negado ou o nome da pessoa for negado no pais - faça:
      st.error("Você não pode entrar no país")                                                      #função do streamlit que gera uma mensagem de erro
      return                                                                                        #retorna a função do streamlit
    st.success("Voce pode entrar no país")                                                          #função do streamlit que gera uma mensagem de sucesso
    return                                                                                          #retorna a função do streamlit


                                
  st.set_page_config(                                                                               #função do streamlit que configura a pagina da web, com as cartacteristicas de titulo e layout
     page_title="France Airport",
     page_icon="🇫🇷",
     layout="centered",
     initial_sidebar_state="expanded",
     menu_items={
         'Get Help': 'https://bit.ly/documentacao-cp-cyber-henrico',
         'Report a bug': "https://www.linkedin.com/in/henricobela/",
         'About': "https://github.com/henricobela/cyber_cp1"
     }
 )

  st.title("Imigration Verifier")                                                                   #função do streamlit que gera um título na pagina da web e da um nome a ele
  form = st.form(key="form")                                                                        #função do streamlit que gera um formulario

  with form:                                                                                        #função para gerar e abrir um formulario na tela da pagina web
    ownerName = st.text_input("Nome do Indíviduo:")                                                 #guarda na variavel ownerName a função do streamlit que gera um campo para se digitar, no caso, o nome do individuo
    passportNumber = st.text_input("País e Número do Passaporte (EX:FRA123456789):")                #guarda na variavel passportNumber a função do streamlit que gera um campo para se digitar, caso o numero de passaporte
    submit = st.form_submit_button(label="SEND")                                                    #guarda na variavel submit a função do streamlit que gera um botão



  if submit:                                                                                        #condição se for apertado o botão
    if not ownerName or not passportNumber:                                                         #condição se não foi digitado o nome OU o passaport no formulario
      st.warning("Digite todos os campos por favor...")                                             #função do streamlit que gera uma mensagem de aviso
    else:                                                                                           #então da condição submit
      try:                                                                                          #função de tentativa para executar a verificação de nome e numero de passaporte
        verifyPerson(ownerName, passportNumber)                                                     #chamada do metodo criado para a verificação
      except:                                                                                       #exceção da tentativa para executar a verificação
        st.error("Você não pode entrar no país")                                                    #tratativa para o erro caso nao sejam digitadas as informações necessárias



if __name__=='__main__':                                                                            #condição para inicializar o programa
  main()                                                                                            #chamada do programa.



Overwriting app.py


In [52]:
!streamlit run app.py & npx localtunnel --port 8501                                                 #chamada para criação da pagina web e inicialização do app.py

2022-05-05 00:43:43.012 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.91.39.136:8501

npx: installed 22 in 2.812s
your url is: https://wide-pots-end-34-91-39-136.loca.lt
  Stopping...
^C
